# 统计方法与机器学习-实验报告1
温兆和 10205501432

## 背景描述
为了调查吃巧克力对心血管健康的影响，实验由三种类型的巧克力组成：100g的黑巧克力，含有200mg全脂牛奶的100g黑巧克力和200g的牛奶巧克力。12个实验对象：7女5男。在不同的天数里，每个实验对象将吃一种类型的巧克力，一个小时后测量他们血浆的总抗氧能力。

## 数据描述
实验次序本身具有随机性，无需再随机化。请使用Project_1.csv中的数据集。
显著性水平α 取 0.05。

## 实验过程
* 两两比较3种巧克力对心血管健康的影响是否存在差异。

我们先通过以下代码引入需要用到的Python库，设定好显著性水平、因子水平数、样本容量等参数，定义好将巧克力类别从1、2、3转为A、B、C的函数，并将数据集中有用的数据（即Chocolate和Capacity）保留，其余没用的删除，为整个实验做好准备工作

In [4]:
import os # 修改工作目录

import numpy as np
import pandas as pd
import scipy.stats as stats # 统计函数
import matplotlib.pyplot as plt
from plotnine import * # ggplot 绘图
from plotnine.data import mpg
from jupyterquiz import display_quiz # Quiz

#from ggplot import ggplot

import math

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import f
from scipy.stats import t
from statsmodels.stats.stattools import durbin_watson
alpha = 0.05 # significant level
a = 3 # number of levels
m = 12 # number of replicates
n = a*m # sample size
def ChangeNumToLetter(a):
    if a==1:
        return 'A'
    elif a==2:
        return 'B'
    else:
        return 'C'
Data = pd.read_csv("./Project/Project_1.csv")
## Construct a New Dataset
Data = Data[['Chocolate','Capacity']] # select some columns from a dataset
Data.columns = ['chokolate','capacity']
print(Data.columns)

Index(['chokolate', 'capacity'], dtype='object')


再把’chokolate’中的1、2、3改为A、B、C

In [5]:
for z in range(0,36):
    Data['chokolate'][z]=ChangeNumToLetter(Data['chokolate'][z])
print(Data)

   chokolate  capacity
0          A     118.8
1          A     122.6
2          A     115.6
3          A     113.6
4          A     119.5
5          A     115.9
6          A     115.8
7          A     115.1
8          A     116.9
9          A     115.4
10         A     115.6
11         A     107.9
12         B     105.4
13         B     101.1
14         B     102.7
15         B      97.1
16         B     101.9
17         B      98.9
18         B     100.0
19         B      99.8
20         B     102.6
21         B     100.9
22         B     104.5
23         B      93.5
24         C     102.1
25         C     105.8
26         C      99.6
27         C     102.7
28         C      98.8
29         C     100.9
30         C     102.8
31         C      98.7
32         C      94.7
33         C      97.8
34         C      99.7
35         C      98.6


/tmp/ipykernel_1377993/3410344499.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


假设吃完第一种巧克力一小时后血浆的总抗氧能力为$μ_1$，吃完第二种巧克力一小时后血浆的总抗氧能力为$μ_2$，则这个检验的原假设和备择假设为
$$𝐻_0: μ_1=μ_2  vs  𝐻_1: μ_1≠μ_2$$

在方差相等的前提下，由于

$$t=\frac{(\bar{x} - \bar{y}-(μ_1-μ_2))\sqrt{m+n-2}}{\sqrt{m^{-1} + n^{-1}}\sqrt{(m-1){S_x}^2+(n-1){S_y}^2}}\sim t(m+n-2)$$

故构造检验统计量

$$t_0 = \frac{ (\bar{x} - \bar{y})}{\sqrt{m^{-1} + n^{-1}}\sqrt{(m-1){S_x}^2+(n-1){S_y}^2}}$$

拒绝域为

$$\{|𝑡_0|>𝑡_{1−𝛼/2}(𝑚+𝑛−2)\}$$

当$t_0$落入拒绝域，则拒绝原假设，认为两种巧克力对心血管健康的影响不一致；否则接受原假设，认为两种巧克力对心血管健康的影响一致。

我们先来比较第一种和第二种巧克力对心血管健康的影响。此时第一组和第二组的样本量$m=n=12$，$\bar{x}$与${S_x}^2$分别是处理后的数据集Data中’chokolate’为’A’时所有’capacity’的均值与方差，$\bar{y}$与${S_y}^2$分别是处理后的数据集Data中’chokolate’为’B’时所有’capacity’的均值与方差。我们通过以下代码提取出数据集Data中关于第一种和第二种巧克力的数据，分别计算出吃完这两种巧克力后血浆的总抗氧能力的均值、方差、样本量等参数并计算出$t_0$和本次检验的拒绝域，再进行比较。


In [7]:
Data1=Data.values
list_type = ['A','B']
Group2_data = [Data1[Data1[:,0] == x,1] for x in list_type]
# Two-sample t test
t0,pVal0 = stats.ttest_ind(Group2_data[0],Group2_data[1])
print("t statistic is:" ,round(t0,4))
print("critical value is：", round(t.ppf(1-alpha/2,12*2-2),4))

t statistic is: 11.1057
critical value is： 2.0739


可见，$t_0=11.1057$，拒绝域为$\{|t_0|>2.0739\}$。故$t_0$落入拒绝域，我们拒绝原假设，认为第一种巧克力和第二种巧克力对心血管健康的影响不一致。

接下来，我们再用同样的方法来比较另外两组巧克力对心血管健康影响的差异。

我们先比较第一种和第三种巧克力：

In [8]:
list_type = ['A','C']
Group2_data = [Data1[Data1[:,0] == x,1] for x in list_type]
# Two-sample t test
t0,pVal0 = stats.ttest_ind(Group2_data[0],Group2_data[1])
print("t statistic is:" ,round(t0,4))
print("critical value is：", round(t.ppf(1-alpha/2,12*2-2),4))

t statistic is: 12.0478
critical value is： 2.0739


$t_0$落入拒绝域，我们拒绝原假设，认为第一种巧克力和第三种巧克力对心血管健康的影响不一致。

再比较第二种和第三种巧克力：

In [11]:
list_type = ['B','C']
Group2_data = [Data1[Data1[:,0] == x,1] for x in list_type]
# Two-sample t test
t0,pVal0 = stats.ttest_ind(Group2_data[0],Group2_data[1])
print("t statistic is:" ,round(t0,4))
print("critical value is：", round(t.ppf(1-alpha/2,12*2-2),4))

t statistic is: 0.4126
critical value is： 2.0739


$t_0$没有落入拒绝域，我们接受原假设，认为第二种巧克力和第三种巧克力对心血管健康的影响一致。

* 判断3种巧克力对心血管健康的影响是否有差异。

在比较3种巧克力对心血管健康的影响的差异时，我我们的响应变量是志愿者吃完某种巧克力一小时后血浆的总抗氧能力，因子有三种可能的取值，每种因子水平下有12个样本，总的样本容量是36。我们令

$$y_{ij} = µ_i + ε_{ij}，其中i\in{1，2，3}，j\in[1,12]\bigcap Z$$

$$µ_i = µ + α_i，其中i\in{1，2，3}$$

可见，$α_i$是各个水平下的效应，且$\sum_{i=1}^{12}α_i=0$，$ε_{ij}$是水平i下第j个样本与该水平样本均值间的偏差。我们发现，总偏差平方和

$$SS_T=12\sum_{i=1}^{3} (\bar{y}_{i\cdot}-\bar{y}_{\cdot\cdot})^2+\sum_{i=1}^{3} \sum_{j=1}^12 (y_{ij} - \bar{y}_{i\cdot})^2$$

其中

$$SS_A=12\sum_{i=1}^{3} (\bar{y}_{i\cdot}-\bar{y}_{\cdot\cdot})^2$$ 

$$SS_E=\sum_{i=1}^{3} \sum_{j=1}^12 (y_{ij} - \bar{y}_{i\cdot})^2$$

$SS_A$表示不同水平下样本的均值与所有样本均值间偏差的平方和，$SS_E$表示某一水平下样本与该水平下样本均值偏差的平方和。对于一组给定的数据，总偏差平方和$SS_T$是确定的。

如果三种巧克力对心血管健康的影响一致，则不同水平对总体均值的效应$α_i$均为0。所以在这个假设检验问题中，原假设和备择假设分别为

$$𝐻_0: α_1=α_2=α_3=0 vs 𝐻_1:\exists i\in{1，2，3}s.t. α_i≠0$$

由卡方分布的可加性，我们容易证明

$$\frac{SS_E}{σ^2}\sim χ^2 (n-a)$$

$$\frac{SS_A}{σ^2} \sim χ^2 (a-1)(H_0成立)$$

由于正态分布中样本均值与样本方差独立，所以

$$\sum_{j=1}^{12} (y_{ij} - \bar{y}_{i\cdot})^2 \bot \bar{y_{i.}}$$

又$SS_A$可被看作$y_{i.}$的函数，所以$SS_A$与$SS_E$独立。故

$$F=\frac{SS_A/(a-1)}{SS_E/(n-a)}\sim F(a-1,n-a)$$
我们把上述$F$作为检验统计量，在显著性水平$α$下，拒绝域为

$$\{F \geq F_{1-α}(a-1,n-a)\}$$

于是，我们就可以通过以下代码打印出这个单因子方差分析假设检验问题的方差分析表。

In [12]:
model = ols('capacity~chokolate', Data).fit() # Add C() to numeric group indices
anovaResults = round(anova_lm(model),4)
print("\nThe ANOVA table: \n", anovaResults)


The ANOVA table: 
              df     sum_sq   mean_sq        F  PR(>F)
chokolate   2.0  1952.6439  976.3219  93.5756     0.0
Residual   33.0   344.3058   10.4335      NaN     NaN


从方差分析表中，我们看到这个假设检验的p值几乎为0，小于显著性水平$α =0.05$，所以我们拒绝原假设，认为这三种巧克力对心血管健康的影响不一致。

* 试说明所使用模型的合理性。

在这个单因子方差分析模型中，

$$y_{ij} = µ + α_i + ε_{ij}，其中i\in{1，2，3}，j\in[1,12]\bigcap Z$$

我们假设

$$ε_{ij} \sim^{i.i.d} N(0,σ^2)$$

所以验证这个模型的合理性就等价于验证εij的独立性、方差齐性和正态性。

首先我们用Durbin-Watson方法检验$ε_{ij}$的独立性。我们假设

$$ε_i=ρε_{i-1}+ω_i$$

其中

$$ω_i\sim^{i.i.d} N(0,σ^2 )$$

当$ρ=0$，则残差序列独立。该检验的检验统计量为

$$DW = \frac{\sum_{i=2}^n (ε_i - ε_{i-1})^2}{\sum_{i=1}^n ε_{i}^2}$$

若$DW$接近于2，则认为残差序列独立。我们通过以下代码计算数据集Data的$DW$值。

In [13]:
Res = Data1
list_type = ['A','B','C']
for i in list_type:
    group = Res[Res[:,0] == i,1]
    Res[Res[:,0] == i,1] = group - np.mean(group)
DW = durbin_watson(Res[:,1])
print("DW statistic is", round(DW,4))

DW statistic is 2.2991


可见，该独立性检验的$DW$值与2比较接近，可以认为残差之间相互独立。

接下来，我们分别用Bartlett方法和修正后的Levene检验来检验$ε_{ij}$的方差齐性。

在Bartlett检验中，检验统计量为

$$\chi_0^2 = 2.3026 \frac{q}{c}$$

其中

$$ q = (n - a) \log_{10}s_p^2 - \sum_{i=1}^a (m_i-1)  \log_{10}s_i^2 $$
$$c = 1 + \frac{1}{3(a-1)}\left( \sum_{i=1}^a (m_i-1)^{-1} - (n-a)^{-1} \right)$$
$$s_p^2 = \frac{\sum_{i}  (m_i-1)s^2_i}{n-a}$$
$$s_i^2 表示第i组数据的样本方差$$

拒绝域为

$$\{\chi_0^2 > \chi_{1-\alpha,a-1}^2\}$$

我们通过以下代码对${χ_0}^2$进行检验。

In [14]:
list_type = ['A','B','C']
Group2_data = [Data1[Data1[:,0] == x,1] for x in list_type]
Bart_stat, Bart_pVal = stats.bartlett(Group2_data[0], Group2_data[1], Group2_data[2])
print("Bartlett's test statistic is", round(Bart_stat,4))
print("The p value is", round(Bart_pVal,4))

Bartlett's test statistic is 0.4247
The p value is 0.8087


可见，${χ_0}^2$没有落入拒绝域，所以我们接受原假设，即诸$ε_{ij}$的方差相等，均为$σ^2$。

而在修正后的Levene检验中，我们通过构建检验统计量

$$y_{ij}^{\ast} = |y_{ij} - \tilde{y}_{i}|,i\in{1,2,3},j\in[1,12]\bigcap Z$$

来比较各个水平下样本与样本中位数的偏差之和是否相等。我们通过以下代码对${y_{ij}}^*$进行检验。

In [15]:
Lev_stat, Lev_pVal = stats.levene(Group2_data[0], Group2_data[1], Group2_data[2])
print("Levene's test statistic is", round(Lev_stat,4))
print("The p value is", round(Lev_pVal,4))

Levene's test statistic is 0.0213
The p value is 0.979


由此，我们接受原假设，即诸$ε_{ij}$的方差相等，均为$σ^2$。

最后，我们再用Shapiro-Wilk方法来验证$ε_{ij}$的正态性。我们构造统计量

$$W = \frac{\left(\sum_{i=1}^n (a_{i}-\bar{a}) (x_{(i)}-\bar{x})\right)^2}{\sum_{i=1}^n (a_i-\bar{a})^2 \sum_{i=1}^n (x_{(i)} - \bar{x})^2}$$

其中$x_{(i)}$是来自正态分布的顺序统计量，$a_i$在给定样本量$n$时是理论分位数，是定值。$\sqrt{W}$就是理论分位数和样本分位数的相关系数。如果理论分位数和样本分位数在一条直线上，就认为$ε_{ij}$服从正态分布。由于相关系数越接近1两随机变量的线性相关性越强，所以这个正态性检验的拒绝域是

$$\{W\leq W_α\}$$

我们通过以下代码来检验$ε_{ij}$的正态性。

In [17]:
Res1 = Res[:,1].astype(float)
SW_stat,SW_pVal = stats.shapiro(Res1)
print("Shapiro-Wilk test statistic is", round(SW_stat,4))
print("The p value is", round(SW_pVal,4))

Shapiro-Wilk test statistic is 0.9625
The p value is 0.2572


可见，统计量的值非常接近于1且没有落入拒绝域，所以我们认为原假设成立，即$ε_{ij}$服从正态分布。

综上所述，我们可以证明这个单因子方差分析模型的假设

$$ε_{ij} \sim^{i.i.d} N(0,σ^2 )$$

成立，所以用该模型比较三种巧克力对心血管健康的影响合理。

* 估计食用这3种巧克力一小时后血浆的总抗氧能力。请分别给出点估计和区间估计。

在这个实验中，待估参数为所有数据的总体均值$μ$。由于

$$y_{ij}\sim N(μ+α_i,σ^2),i\in{1,2,3},j\in[1,12]\bigcap Z$$

所以我们可以写出似然函数

$$L(μ,α_1,α_2,α_3,σ^2 )=\prod_{i=1}^3\prod_{j=1}^{12}{\frac{1}{σ\sqrt{2\pi}}exp\{-\frac{(y_{ij}-μ-α_i)^2}{2σ^2}\}}$$

取对数并对$μ$求偏导，得

$$\frac{∂l}{∂μ}=\frac{1}{σ^2} \sum_{i=1}^3\sum_{j=1}^{12}(y_{ij}-μ-α_i)$$

$$\frac{∂l}{∂α_i}=\frac{1}{σ^2} \sum_{j=1}^{12}(y_{ij}-μ-α_i)$$

$$\frac{∂l}{∂σ^2}={-18\frac{1}{σ^2}}+{\frac{\sum_{i=1}^3\sum_{j=1}^{12}(y_{ij}-μ-α_i)^2}{2σ^4}}$$

令$\frac{∂l}{∂μ}=\frac{∂l}{∂α_i}=\frac{∂l}{∂σ^2}=0$，又因为$\sum_{i=1}^3 α_i=0$，我们通过解方程组得到$μ$的最大似然估计为

$$\hat{μ}=\bar{y_{..}}$$

我们通过以下代码求$μ$的最大似然估计。

In [18]:
hat_mu = np.mean(Data.capacity)
print(type(hat_mu))
print("Overall Mean is:", round(hat_mu,4))

<class 'float'>
Overall Mean is: 105.6472


于是有

$$\hat{μ}=105.6472$$

接下来，我们再来求$μ$的区间估计。

由于

$$\bar{y_{..}}=\frac{1}{36}\sum_{i=1}^3\sum_{j=1}^{12}y_{ij}=μ+\bar{ε_{..}}\sim N(μ,\frac{σ^2}{36})$$

所以

$$\frac{(\bar{y_{..}}-μ)}{σ\sqrt{\frac{1}{36}}}\sim N(0,1)$$

又

$$\frac{SS_E}{σ^2}\sim χ^2 (33)$$

所以

$$\frac{\sqrt{36}(\bar{y_{..}}-μ)}{\sqrt{\frac{SS_E}{33}}}\sim t(33)$$

由此，我们可以得到$μ$的置信水平为$1-α$的置信区间为

$$[\bar{y_{..}}-t_{1-α/2} (33){\sqrt{\frac{SS_E}{{33}\times{36}}}},\bar{y_{..}}+t_{1-α/2} (33){\sqrt{\frac{SS_E}{{33}\times{36}}}}]$$

从4.2中的方差分析表中可以看出，$SS_E=344.3058$。

我们通过以下代码来计算这个区间估计的置信下限和置信上限。

In [19]:
tt=round(t.ppf(1-alpha/2,33),4)
totalaverage=0
for z in range(0,36):
    totalaverage=totalaverage+Data['capacity'][z]
totalaverage=totalaverage/36
lower = totalaverage-tt*((344.3058/1188)**0.5)
upper = totalaverage+tt*((344.3058/1188)**0.5)
print(round(lower,4))
print(round(upper,4))

104.552
106.7425


所以，$μ$的置信水平为0.95的置信区间为

$$[104.552,106.7425]$$
## 总结

在本周的实验中，我们复习了二样本t检验的知识，还首次尝试通过方差分析的均值模型和效应模型来比较多组方差相等的正态总体的均值，学会通过验证模型的假设来判断模型的合理性，还学会了如何求维度较高的统计量的最大似然估计，为以后的学习打好了基础。